In [3]:
!pip install yfinance

StatementMeta(spark001, 13, 2, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.0 MB/s eta 0:00:00:00:01
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for peewee: filename=peewee-3.17.5-cp310-cp310-linux_x86_64.whl size=275237 sha256=7f056098a76a1f12ac52a08c75ce82157d10854831591fdac2354bdc5787de29
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/06/80/9b/98db0d58349a2f5c09f8406789ade4270762f97b7d26f2fa22
Successfully built peewee


In [4]:
import yfinance as yf
from pyspark.sql.types import *
from pyspark.sql import functions as f
import requests
import pandas as pd 
import datetime
from notebookutils import mssparkutils

StatementMeta(spark001, 13, 3, Finished, Available)

In [38]:
def get_stock_price(symbol_list,start_date,end_date):
    historical_data = {}
    for symbol in symbol_list:
        ticker = yf.Ticker(symbol)
        data = ticker.history(start=start_date, end=end_date)
        historical_data[symbol] = data

    return historical_data

def transform_df(historical_data,symbol_list):
    dfs=[]
    for symbol in symbol_list:
        df = pd.DataFrame(historical_data[symbol])
        df['symbol']= symbol
        dfs.append(df)
    df = pd.concat(dfs).reset_index()
    df['Date'] = df['Date'].dt.date
    return df 

StatementMeta(spark001, 13, 37, Finished, Available)

In [39]:
if __name__ == "__main__":

    symbol_list=['NVDA','AMD','INTC','QCOM','GOOG','MSFT','AMZN','AAPL']
    
    end_date = datetime.datetime.now()
    start_date = end_date-datetime.timedelta(days=2*365)
    historical_data = get_stock_price(symbol_list,start_date,end_date)

    pdf = transform_df(historical_data,symbol_list)
    sp_df = spark.createDataFrame(pdf)
    file_path ='abfss://files@datalake9vbgk2l.dfs.core.windows.net/synapse/workspaces/data/stock_price'
    sp_df.write.option("header", True).csv(file_path,mode='overwrite')

StatementMeta(spark001, 13, 38, Finished, Available)